In [1]:
%load_ext autoreload
%autoreload 2

# AutoModel

In [31]:
from glasses.nn.models.AutoModel import AutoModel
from glasses.nn.models.AutoConfig import AutoConfig
import torch

In [32]:
AutoModel.from_name('resnet50').summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
         Conv2dPad-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
         Conv2dPad-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
        Conv2dPad-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
        Conv2dPad-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

(tensor(25557032), tensor(25557032), tensor(97.4923), tensor(439.0076))

In [27]:
model = AutoModel.from_name('resnet18')
# notify to the model we want to have the features
model.encoder.features
model(torch.rand((1,3,224,224)))
# get the features :)
features = model.encoder.features
[print(f.shape) for f in features]

NameError: name 'AutoModel' is not defined

In [ ]:
AutoConfig.from_name('unet')

# RegNet

In [24]:
import timm
from glasses.nn.models.classification.regnet import RegNet
regnet_timm = timm.create_model("regnetx_004")

regnet_glasses = RegNet.regnetx_004()

In [25]:
regnet_glasses.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
         Conv2dPad-4         [-1, 32, 112, 112]           1,024
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              ReLU-6         [-1, 32, 112, 112]               0
         Conv2dPad-7           [-1, 32, 56, 56]           4,608
       BatchNorm2d-8           [-1, 32, 56, 56]              64
              ReLU-9           [-1, 32, 56, 56]               0
        Conv2dPad-10           [-1, 32, 56, 56]           1,024
      BatchNorm2d-11           [-1, 32, 56, 56]              64
        Conv2dPad-12           [-1, 32, 56, 56]           1,024
      BatchNorm2d-13           [-1, 32, 56, 56]              64
    ResNetShorcut-14           [-1, 32,

(tensor(5157512), tensor(5157512), tensor(19.6743), tensor(102.1916))

In [26]:
from torchsummary import summary


summary(regnet_timm.cuda(), (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
              ReLU-2         [-1, 32, 112, 112]               0
    BatchNormAct2d-3         [-1, 32, 112, 112]              64
         ConvBnAct-4         [-1, 32, 112, 112]               0
            Conv2d-5         [-1, 32, 112, 112]           1,024
              ReLU-6         [-1, 32, 112, 112]               0
    BatchNormAct2d-7         [-1, 32, 112, 112]              64
         ConvBnAct-8         [-1, 32, 112, 112]               0
            Conv2d-9           [-1, 32, 56, 56]           4,608
             ReLU-10           [-1, 32, 56, 56]               0
   BatchNormAct2d-11           [-1, 32, 56, 56]              64
        ConvBnAct-12           [-1, 32, 56, 56]               0
           Conv2d-13           [-1, 32, 56, 56]           1,024
   BatchNormAct2d-14           [-1, 32,

(tensor(5157512), tensor(5157512), tensor(19.6743), tensor(121.1006))

# UNet

In [ ]:
import torch
from glasses.nn.models.classification import *
from glasses.utils.Storage import ForwardModuleStorage
from glasses.nn.models.segmentation.unet import *
from glasses.nn.models.classification.resnet import ResNetEncoder, ResNet, ResNetBottleneckBlock
from glasses.nn.models.classification.efficientnet import *
from glasses.nn.models.classification import *
from torch import nn

In [ ]:
unet = UNet.from_encoder(lambda *args, **kwargs: ResNet.resnet18(*args, **kwargs))

unet

In [ ]:

unet = UNet.from_encoder(partial(AutoModel.from_name, 'resnet18'))
model.summary(input_shape=(1,384,384))

# Benchmark

In [ ]:
import timm
from glasses.nn.models import *
timm.create_model("regnetx_008")

In [ ]:
SEResNet.cse_resnet50()

In [ ]:
enc = EfficientNet.efficientnet_b0().encoder
x = torch.randn((1,3,224,224))

In [ ]:
from glasses.nn.models.classification.efficientnet import *
import timm

from transfer_weights import clone_model
from benchmark import benchmark

src = timm.create_model('tf_efficientnet_lite1', pretrained='True')
dst = EfficientNetLite.efficientnet_lite1(mode='same')

In [ ]:
def from_encoder(model_def, *args, **kwargs):
    model = model_def( *args, **kwargs)
    encoder = nn.Identity()
    if isinstance(model, ResNet):
        encoder = WithFeatures(model.encoder, 
                              stages = [
                                  model.encoder.stem[-2], 
                                  *model.encoder.layers,
                              ],
                              features_widths = model.encoder.widths[1:]
                             )
        
    elif isinstance(model, EfficientNet):
        encoder = WithFeatures(model.encoder, 
                              stages = [
                                  model.encoder.stem[-2], 
                                  model.encoder.layers[1],
                                  model.encoder.layers[2],
                                  model.encoder.layers[3],
                              ],
                              features_widths = [model.encoder.widths[0],
                                                 model.encoder.widths[2],
                                                 model.encoder.widths[3],
                                                 model.encoder.widths[4],
                                                ]
                             )
        
    return encoder

In [ ]:
enc = ResNet.resnet18().encoder

In [ ]:
enc.features_widths()

In [ ]:
enc.stages()

In [ ]:
enc.features

enc(x)

enc.features


In [ ]:
m

In [ ]:
backbone = EfficientNet.efficientnet_b0().encoder


encoder = WithFeatures(backbone, 
                              stages = [
                                  backbone.stem[-2], 
                                  backbone.layers[1],
                                  backbone.layers[2],
                                  backbone.layers[3],
                              ],
                              features_widths = [32, 24, 40, 80]
                             )


m = UNet(encoder = lambda *args, **kwargs: encoder, decoder = partial(UNetDecoder, 
                                                                      widths=[1280, 256, 128, 64, 32, 16]))

In [ ]:
enc.features

In [ ]:
name = 'resnet-asdsa'

name.startswith('resnet')

In [ ]:
m.summary((3, 224, 224))

In [ ]:
dst = clone_model(src, dst)

In [ ]:
dst.summary()

In [ ]:
dst

In [ ]:
# import torch

# transform = dst.configs['efficientnet_lite1'].transform

# benchmark(dst.cuda(), transform)

In [ ]:
# benchmark(src.cuda(), transform)

In [ ]:
src = timm.create_model('efficientnet_b0', pretrained='True')
src

In [ ]:
dst.summary()